# Historic Admissions Data

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from pathlib import Path

In [2]:
today = date.today()
today_str = today.strftime("%Y%m%d")
print(today)
print(today_str)


2020-12-23
20201223


In [3]:
if today<(dt.date(today.year, 9, 1)):
    acad_yr = today.year
else:
    acad_yr = today.year + 1

print(f"acad_yr={acad_yr:d}")

acad_yr=2021


In [4]:
#data_path = Path(r"C:\JW\Python\Admissions\funnel")
data_path = Path(r"\\psc-data\E\Applications\Admissions\funnel")
data_store = data_path / "data/stage_data"

df = pd.read_hdf(data_store, key="weekly", mode='r')
df = df[(df["year_term"] > "2011.Spring")]


In [5]:
df.head()

,year_term,stage,PEOPLE_CODE_ID,00,01,02,03,04,05,06,...,45,46,47,48,49,50,51,52,53,curriculum
1443,2012.Fall,Applied,P000012781,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1444,2012.Fall,Applied,P000021989,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,NaN
1445,2012.Fall,Applied,P000023258,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1446,2012.Fall,Applied,P000023557,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1447,2012.Fall,Applied,P000024505,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,FWSW


In [6]:
df.loc[(df['PEOPLE_CODE_ID']=='P000000564')]

,year_term,stage,PEOPLE_CODE_ID,00,01,02,03,04,05,06,...,45,46,47,48,49,50,51,52,53,curriculum


In [7]:
b_yr = 2012
e_yr = 2021
term = 'Fall'

terms = [f"{y}.{term}" for y in range(b_yr, e_yr+1)]
print(terms)
stages = [ 'Applied', 'Accepted', 'Deposited']


['2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', '2019.Fall', '2020.Fall', '2021.Fall']


In [8]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()


In [9]:
print(summ.shape)
summ.head()


(60, 54)


00  01  02  03  04  05  06  07  08  09  ...   44   45  \
year_term   stage                                              ...             
2012.Fall   Accepted    0   0   0   0   0   0   1   1   1   1  ...  396  390   
            Applied     0   0   0   0   0   0   1   1   1   1  ...  508  500   
            Deposited   0   0   0   0   0   0   1   1   1   1  ...  377  379   
2012.Spring Accepted    0   0   0   0   0   0   0   0   1   1  ...    2    2   
            Applied     0   0   0   0   1   1   1   1   1   1  ...    4    4   

                        46   47   48   49   50   51   52   53  
year_term   stage                                              
2012.Fall   Accepted   355  358  354  310  273  265  262  261  
            Applied    450  449  449  403  362  344  340  338  
            Deposited  378  382  383  381  373  364  364  364  
2012.Spring Accepted     2    2    2    2    2    2    2    2  
            Applied      4    4    4    4    4    4    4    4  

[5 rows x 54 columns]

In [10]:
summ_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                0       0         0           0       0         0        17   
01                0       0         0           0       0         0        18   
02                0       0         0           0       0         0        21   
03                0       0         0           0       0         0        25   
04                0       0         0           0       1         0        39   

year_term                   2013.Spring  ... 2020.Fall 2020.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00             86         6           8  ...        10           2       7   
01             96         7           8  ...        10           3       9   
02            114        10          15  ...        10           3      10   
03            129        10          16  ...        10           3      11   
04            153        10          20  ...        10           3      12   

year_term           2021.Fall                   2021.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00               12        22      46        19          20      42        17  
01               11        22      50        17          20      45        17  
02               11        22      55        17          22      50        17  
03               11        22      65        18          24      52        19  
04               11        23      74        18          25      52        19  

[5 rows x 60 columns]

In [11]:
def date_diff_weeks(start, end):
    diff = (pd.to_datetime(end) - pd.to_datetime(start))
    return int( diff / np.timedelta64(1,'W'))

def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    week_number = d.isocalendar()[1]

    if d >= date(year, 9, 1):
        adm_start = date(year, 9, 1)
    else:
        adm_start = date(year - 1, 9, 1)

    adm_week_number = min(date_diff_weeks(adm_start, d), 53)

    return week_number, adm_week_number


In [12]:
def adm_week_end_date(awn, adm_yr):
    """
    returns calendar date for the Friday of the given Admissions Week Number, awn
        and academic year, adm_yr
    """
    
    adm_start_date = dt.date((adm_yr-1), 9, 1)
    
    d = adm_start_date + dt.timedelta(weeks = awn)
    
    year = d.year
    wn = d.isocalendar()[1]

    #week_end = dt.datetime.strptime(f'{year}-{wn}-0', "%Y-%W-%w").date() - dt.timedelta(weeks=1)
    week_end = d + dt.timedelta(days=5 - d.weekday())
    #print(awn, adm_yr, d, year, wn, week_end)
    
    return(week_end)


In [13]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()
summ_t = summ_t.reset_index().rename(columns={"index": "week_num"})
summ_t["wn"] = summ_t.loc[:, ["week_num"]].astype(int)
summ_t.head()

year_term week_num 2012.Fall                   2012.Spring                    \
stage               Accepted Applied Deposited    Accepted Applied Deposited   
0               00         0       0         0           0       0         0   
1               01         0       0         0           0       0         0   
2               02         0       0         0           0       0         0   
3               03         0       0         0           0       0         0   
4               04         0       0         0           0       1         0   

year_term 2013.Fall                    ... 2020.Spring                    \
stage      Accepted Applied Deposited  ...    Accepted Applied Deposited   
0                17      86         6  ...           2       7        12   
1                18      96         7  ...           3       9        11   
2                21     114        10  ...           3      10        11   
3                25     129        10  ...           3      11        11   
4                39     153        10  ...           3      12        11   

year_term 2021.Fall                   2021.Spring                   wn  
stage      Accepted Applied Deposited    Accepted Applied Deposited     
0                22      46        19          20      42        17  0  
1                22      50        17          20      45        17  1  
2                22      55        17          22      50        17  2  
3                22      65        18          24      52        19  3  
4                23      74        18          25      52        19  4  

[5 rows x 62 columns]

In [14]:
week_end = (
    lambda r: adm_week_end_date(int(r["wn"]), acad_yr)
)
summ_t["Week_End"] = summ_t.apply(week_end, axis=1)


In [15]:
summ_t = summ_t.drop(columns=["wn", "week_num"], axis=1).set_index("Week_End")

C:\Users\jwalton\.conda\envs\py38jl\lib\site-packages\pandas\core\generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [16]:
summ_t.head()

year_term  2012.Fall                   2012.Spring                    \
stage       Accepted Applied Deposited    Accepted Applied Deposited   
Week_End                                                               
2020-09-05         0       0         0           0       0         0   
2020-09-12         0       0         0           0       0         0   
2020-09-19         0       0         0           0       0         0   
2020-09-26         0       0         0           0       0         0   
2020-10-03         0       0         0           0       1         0   

year_term  2013.Fall                   2013.Spring  ... 2020.Fall 2020.Spring  \
stage       Accepted Applied Deposited    Accepted  ... Deposited    Accepted   
Week_End                                            ...                         
2020-09-05        17      86         6           8  ...        10           2   
2020-09-12        18      96         7           8  ...        10           3   
2020-09-19        21     114        10          15  ...        10           3   
2020-09-26        25     129        10          16  ...        10           3   
2020-10-03        39     153        10          20  ...        10           3   

year_term                    2021.Fall                   2021.Spring          \
stage      Applied Deposited  Accepted Applied Deposited    Accepted Applied   
Week_End                                                                       
2020-09-05       7        12        22      46        19          20      42   
2020-09-12       9        11        22      50        17          20      45   
2020-09-19      10        11        22      55        17          22      50   
2020-09-26      11        11        22      65        18          24      52   
2020-10-03      12        11        23      74        18          25      52   

year_term             
stage      Deposited  
Week_End              
2020-09-05        17  
2020-09-12        17  
2020-09-19        17  
2020-09-26        19  
2020-10-03        19  

[5 rows x 60 columns]

In [17]:
print(today_str)
writer = pd.ExcelWriter(
    f"{term}_HistoricAdmissionsData_{b_yr:d}-{e_yr:d}_{today_str}.xlsx"
)
for st in stages:
    df_sumt = summ_t.loc[:, (terms, st)]
    #df_sumt.T.reset_index().T.to_excel(writer, st)
    df_sumt.to_excel(writer, st)

writer.save()


20201223
